<br>
This is exploring and modifying PyTorch/Pyro's bayesian regression capabilities; heavily borrowing from their provided example notebook, available at: http://pyro.ai/examples/bayesian_regression.html

I'm going to scale up the toy data set size, for learning purposes. Namely, the number of features (1 -> 10) and the number of data points (100 -> 1000)

<br>
# 1. Regular regression part
## Importing the libraries

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

<br>
## Defining functions and parameters for creating an experimenting data set

In [2]:
N = 1000  # size of toy data

def build_linear_dataset(N, p = 10, noise_std=0.01):
    X = np.random.rand(N, p)
    # w = 3
    w = 3 * np.ones(p)
    # b = 1
    y = np.matmul(X, w) + np.repeat(1, N) + np.random.normal(0, noise_std, size=N)
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return data

<br>
## Defining the simple regression model

In [3]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)

    def forward(self, x):
        return self.linear(x)

regression_model = RegressionModel(10)

<br>
## Defining the training parameters & function, then training
Also printing loss and learned parameters

I bumped:
1. the number of training epochs from 1000 -> 10000
2. the printing freq from 50 -> 250

In [4]:
# Defining loss
loss_fn = torch.nn.MSELoss(size_average = False)
# Defining optimizer
optim = torch.optim.Adam(regression_model.parameters(), lr = 0.05)
# Defining training length
num_iterations = 10000 if not smoke_test else 2

# Defining function to train
def main():
    # Generating the data
    data = build_linear_dataset(N)
    # Defining X
    x_data = data[:, :-1]
    # Defining Y
    y_data = data[:, -1]
    # Training loop
    for j in range(num_iterations):
        # run the model forward on the data
        y_pred = regression_model(x_data).squeeze(-1)
        # calculate the mse loss
        loss = loss_fn(y_pred, y_data)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        # Print loss every 50 iterations
        if (j + 1) % 250 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))
    # Inspect learned parameters
    print("\nLearned parameters:")
    for name, param in regression_model.named_parameters():
        print("%s: " % (name) + str(param.data.numpy()))

if __name__ == '__main__':
    main()

[iteration 0250] loss: 107.2048
[iteration 0500] loss: 86.0182
[iteration 0750] loss: 65.5180
[iteration 1000] loss: 47.1410
[iteration 1250] loss: 31.7164
[iteration 1500] loss: 19.7980
[iteration 1750] loss: 11.3895
[iteration 2000] loss: 6.0006
[iteration 2250] loss: 2.8833
[iteration 2500] loss: 1.2704
[iteration 2750] loss: 0.5322
[iteration 3000] loss: 0.2376
[iteration 3250] loss: 0.1367
[iteration 3500] loss: 0.1076
[iteration 3750] loss: 0.1006
[iteration 4000] loss: 0.0993
[iteration 4250] loss: 0.0991
[iteration 4500] loss: 0.0991
[iteration 4750] loss: 0.0991
[iteration 5000] loss: 0.0991
[iteration 5250] loss: 0.0991
[iteration 5500] loss: 0.0991
[iteration 5750] loss: 0.0991
[iteration 6000] loss: 0.0991
[iteration 6250] loss: 0.0991
[iteration 6500] loss: 0.0991
[iteration 6750] loss: 0.0991
[iteration 7000] loss: 0.0991
[iteration 7250] loss: 0.0991
[iteration 7500] loss: 0.0991
[iteration 7750] loss: 0.0991
[iteration 8000] loss: 0.0991
[iteration 8250] loss: 0.0991
[i

The true (generated) parameters were 3 and 1, with a noise of $\sigma = 0.01$

**Possible typos in Pyro documentation**:

1. They say that $\sigma = 0.1$
2. They say they used a learning rate of $0.01$
2. They say they train for $500$ iterations

None of the above seem to agree with their code

# 2. Bayesian part

Minimally modified from Pyro's documentation:

`random_module()` effectively takes a given `nn.Module` (from PyTorch) and turns it into a distribution over the same module; in our case, this will be a distribution over regressors. Specifically, each parameter in the original regression model is sampled from the provided prior. This allows us to repurpose vanilla regression models for use in the Bayesian setting.

## Toy example - simpler than the next header's

In [5]:
# Defining a standard normal distribution prior
# We need to feed the distribution tensors, rather than ints/floats
loc = torch.zeros(1, 1)
scale = torch.ones(1, 1)
# Recall that "Normal" is from pyro.distributions
prior = Normal(loc, scale)

# overload the parameters in the regression module with samples from the prior
Bayesian_Model = pyro.random_module("abitrary_name",
                                     regression_model,     # The PyTorch model
                                     prior)                # The Pyro prior
# sample a regressor from the prior
sampled_reg_model = Bayesian_Model()

<br>
## Defining the model, which we will train
Another possible code comment typo: their comments say unit normal

In [6]:
def model(data):
    
    # Create wide normal priors - mu = 0, sigma = 10
    # For the weight, defining mu and sigma
    loc1, scale1 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc2, scale2 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc3, scale3 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc4, scale4 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc5, scale5 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc6, scale6 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc7, scale7 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc8, scale8 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc9, scale9 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    loc10, scale10 = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    # For the bias, defining mu and sigma
    bias_loc, bias_scale = torch.zeros(1), 10 * torch.ones(1)
    # For the weight, creating and assigning the prior distribution
    w_prior1 = Normal(loc1, scale1).independent(1)
    w_prior2 = Normal(loc2, scale2).independent(1)
    w_prior3 = Normal(loc3, scale3).independent(1)
    w_prior4 = Normal(loc4, scale4).independent(1)
    w_prior5 = Normal(loc5, scale5).independent(1)
    w_prior6 = Normal(loc6, scale6).independent(1)
    w_prior7 = Normal(loc7, scale7).independent(1)
    w_prior8 = Normal(loc8, scale8).independent(1)
    w_prior9 = Normal(loc9, scale9).independent(1)
    w_prior10 = Normal(loc10, scale10).independent(1)
    # For the bias, creating and assigning the prior distribution
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    
    # Putting those into a dictionary
    priors = {'linear.weight1': w_prior1,
              'linear.weight2': w_prior2,
              'linear.weight3': w_prior3,
              'linear.weight4': w_prior4,
              'linear.weight5': w_prior5,
              'linear.weight6': w_prior6,
              'linear.weight7': w_prior7,
              'linear.weight8': w_prior8,
              'linear.weight9': w_prior9,
              'linear.weight10': w_prior10,
              'linear.bias': b_prior}
    
    # lift module parameters to random variables sampled from the priors
    # This is like creating the nn.Module (analogy to PyTorch)
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a regressor (which also samples w and b)
    # This is like calling the nn.Module (analogy to PyTorch)
    lifted_reg_model = lifted_module()
    
    with pyro.iarange("map", N):
        x_data = data[:, :-1]
        y_data = data[:, -1]

        # run the regressor forward conditioned on data
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, 0.1 * torch.ones(data.size(0))),
                    obs=y_data)

In [7]:
softplus = torch.nn.Softplus()

def guide(data):
    # define our variational parameters
    w_loc1 = torch.randn(1, 1)
    w_loc2 = torch.randn(1, 1)
    w_loc3 = torch.randn(1, 1)
    w_loc4 = torch.randn(1, 1)
    w_loc5 = torch.randn(1, 1)
    w_loc6 = torch.randn(1, 1)
    w_loc7 = torch.randn(1, 1)
    w_loc8 = torch.randn(1, 1)
    w_loc9 = torch.randn(1, 1)
    w_loc10 = torch.randn(1, 1)
    # note that we initialize our scales to be pretty narrow
    w_log_sig1 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig2 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig3 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig4 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig5 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig6 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig7 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig8 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig9 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    w_log_sig10 = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    b_loc = torch.randn(1)
    b_log_sig = torch.tensor(-3.0 * torch.ones(1) + 0.05 * torch.randn(1))
    # register learnable params in the param store
    mw_param1 = pyro.param("guide_mean_weight1", w_loc1)
    sw_param1 = softplus(pyro.param("guide_log_scale_weight1", w_log_sig1))
    mw_param2 = pyro.param("guide_mean_weight2", w_loc2)  
    sw_param2 = softplus(pyro.param("guide_log_scale_weight2", w_log_sig2))
    mw_param3 = pyro.param("guide_mean_weight3", w_loc3)
    sw_param3 = softplus(pyro.param("guide_log_scale_weight3", w_log_sig3))
    mw_param4 = pyro.param("guide_mean_weight4", w_loc4)
    sw_param4 = softplus(pyro.param("guide_log_scale_weight4", w_log_sig4))
    mw_param5 = pyro.param("guide_mean_weight5", w_loc5)
    sw_param5 = softplus(pyro.param("guide_log_scale_weight5", w_log_sig5))
    mw_param6 = pyro.param("guide_mean_weight6", w_loc6)
    sw_param6 = softplus(pyro.param("guide_log_scale_weight6", w_log_sig6))
    mw_param7 = pyro.param("guide_mean_weight7", w_loc7)
    sw_param7 = softplus(pyro.param("guide_log_scale_weight7", w_log_sig7))
    mw_param8 = pyro.param("guide_mean_weight8", w_loc8)
    sw_param8 = softplus(pyro.param("guide_log_scale_weight8", w_log_sig8))
    mw_param9 = pyro.param("guide_mean_weight9", w_loc9)
    sw_param9 = softplus(pyro.param("guide_log_scale_weight9", w_log_sig9))
    mw_param10 = pyro.param("guide_mean_weight10", w_loc10)
    sw_param10 = softplus(pyro.param("guide_log_scale_weight10", w_log_sig10))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    # guide distributions for w and b
    w_dist1 = Normal(mw_param1, sw_param1).independent(1)
    w_dist2 = Normal(mw_param2, sw_param2).independent(1)
    w_dist3 = Normal(mw_param3, sw_param3).independent(1)
    w_dist4 = Normal(mw_param4, sw_param4).independent(1)
    w_dist5 = Normal(mw_param5, sw_param5).independent(1)
    w_dist6 = Normal(mw_param6, sw_param6).independent(1)
    w_dist7 = Normal(mw_param7, sw_param7).independent(1)
    w_dist8 = Normal(mw_param8, sw_param8).independent(1)
    w_dist9 = Normal(mw_param9, sw_param9).independent(1)
    w_dist10 = Normal(mw_param10, sw_param10).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weight1': w_dist1,
             'linear.weight2': w_dist2,
             'linear.weight3': w_dist3,
             'linear.weight4': w_dist4,
             'linear.weight5': w_dist5,
             'linear.weight6': w_dist6,
             'linear.weight7': w_dist7,
             'linear.weight8': w_dist8,
             'linear.weight9': w_dist9,
             'linear.weight10': w_dist10,
             'linear.bias': b_dist}
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

In [8]:
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [9]:
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N)
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if (j) % 250 == 0:
            print("[iteration %04f] loss: %.4f" % ((j+1), loss / float(N)))

if __name__ == '__main__':
    main()

[iteration 1.000000] loss: 162.1105
[iteration 251.000000] loss: -0.9727
[iteration 501.000000] loss: -1.3658
[iteration 751.000000] loss: -1.3460
[iteration 1001.000000] loss: -1.3704
[iteration 1251.000000] loss: -1.3631
[iteration 1501.000000] loss: -1.3712
[iteration 1751.000000] loss: -1.3580
[iteration 2001.000000] loss: -1.3712
[iteration 2251.000000] loss: -1.3588
[iteration 2501.000000] loss: -1.3596
[iteration 2751.000000] loss: -1.3706
[iteration 3001.000000] loss: -1.3650
[iteration 3251.000000] loss: -1.3714
[iteration 3501.000000] loss: -1.3643
[iteration 3751.000000] loss: -1.3691
[iteration 4001.000000] loss: -1.3688
[iteration 4251.000000] loss: -1.3639
[iteration 4501.000000] loss: -1.3530
[iteration 4751.000000] loss: -1.3612
[iteration 5001.000000] loss: -1.3574
[iteration 5251.000000] loss: -1.3429
[iteration 5501.000000] loss: -1.3681
[iteration 5751.000000] loss: -1.3680
[iteration 6001.000000] loss: -1.3677
[iteration 6251.000000] loss: -1.3671
[iteration 6501.0

In [10]:
for name in pyro.get_param_store().get_all_param_names():
    print("[%s]: " % (name) + str(pyro.param(name).data.numpy()))

[guide_mean_weight1]: [[0.38164145]]
[guide_log_scale_weight1]: [[-2.9783652]]
[guide_mean_weight2]: [[-0.09637374]]
[guide_log_scale_weight2]: [[-3.0471041]]
[guide_mean_weight3]: [[-1.396597]]
[guide_log_scale_weight3]: [[-3.007582]]
[guide_mean_weight4]: [[0.8299429]]
[guide_log_scale_weight4]: [[-3.0204463]]
[guide_mean_weight5]: [[-1.4282409]]
[guide_log_scale_weight5]: [[-3.0169454]]
[guide_mean_weight6]: [[-1.0564986]]
[guide_log_scale_weight6]: [[-3.0394912]]
[guide_mean_weight7]: [[0.33928502]]
[guide_log_scale_weight7]: [[-2.9134922]]
[guide_mean_weight8]: [[0.5257805]]
[guide_log_scale_weight8]: [[-2.9970658]]
[guide_mean_weight9]: [[0.46341905]]
[guide_log_scale_weight9]: [[-3.0564148]]
[guide_mean_weight10]: [[0.043645]]
[guide_log_scale_weight10]: [[-2.9418678]]
[guide_mean_bias]: [1.0009319]
[guide_log_scale_bias]: [-5.9051294]
[module$$$linear.weight]: [[2.9998455 3.0000665 2.998335  2.9987602 3.000027  2.997674  2.9994624
  2.9990997 2.9985225 3.0002964]]


In [11]:
np.linspace(6, 7, num=20)

array([6.        , 6.05263158, 6.10526316, 6.15789474, 6.21052632,
       6.26315789, 6.31578947, 6.36842105, 6.42105263, 6.47368421,
       6.52631579, 6.57894737, 6.63157895, 6.68421053, 6.73684211,
       6.78947368, 6.84210526, 6.89473684, 6.94736842, 7.        ])

In [12]:
X1 = np.linspace(6, 7, num=20)
X2 = np.linspace(6, 7, num=20)
X3 = np.linspace(6, 7, num=20)
X4 = np.linspace(6, 7, num=20)
X5 = np.linspace(6, 7, num=20)
X6 = np.linspace(6, 7, num=20)
X7 = np.linspace(6, 7, num=20)
X8 = np.linspace(6, 7, num=20)
X9 = np.linspace(6, 7, num=20)
X10 = np.linspace(6, 7, num=20)


y = (3 * X1) + (3 * X2) +(3 * X3) +(3 * X4) +(3 * X5) +(3 * X6) +(3 * X7) +(3 * X8) +(3 * X9) +(3 * X10) + 1
X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, y = X1.reshape((20, 1)), X2.reshape((20, 1)), X3.reshape((20, 1)), X4.reshape((20, 1)), X5.reshape((20, 1)), X6.reshape((20, 1)), X7.reshape((20, 1)), X8.reshape((20, 1)), X9.reshape((20, 1)), X10.reshape((20, 1)), y.reshape((20, 1))
x_data1, x_data2, x_data3, x_data4, x_data5, x_data6, x_data7, x_data8, x_data9, x_data10, y_data = torch.tensor(X1).type(torch.Tensor), torch.tensor(X2).type(torch.Tensor), torch.tensor(X3).type(torch.Tensor), torch.tensor(X4).type(torch.Tensor), torch.tensor(X5).type(torch.Tensor), torch.tensor(X6).type(torch.Tensor), torch.tensor(X7).type(torch.Tensor), torch.tensor(X8).type(torch.Tensor), torch.tensor(X9).type(torch.Tensor), torch.tensor(X10).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
loss = nn.MSELoss()
y_preds = torch.zeros(20, 1)
for i in range(20):
    # guide does not require the data
    sampled_reg_model = guide(None)
    
    x_data = torch.cat((x_data1, x_data2, x_data3, x_data4, x_data5,
                        x_data6, x_data7, x_data8, x_data9, x_data10),
                       dim = 1)
    
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / 20
print ("Loss: ", loss(y_preds, y_data).item())

Loss:  0.002657769713550806
